# 交差検証時の警告について

K-fold 交差検証時に出力された、以下の警告がとても気になっているため、為念で調査させていただいております。

---
Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.

---

結論としては、クラスにおけるサンプル数が、K-fold の分割数（デフォルト＝３件）を下回った場合、評価ルール適用結果（マイオペの場合はAccuracy）の保証ができない・・・といったところでしょうか。

## (1) テストデータ／環境準備

In [1]:
'''
    プロトタイピング用のパスと、Botライブラリーパスを取得／設定します
'''
import sys
import os

prototype_dir = os.path.join(os.getcwd(), '..')
prototype_dir = os.path.abspath(prototype_dir)

learning_dir = os.path.join(prototype_dir, '..')
learning_dir = os.path.abspath(learning_dir)
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

print('prototype_dir=%s\nlearning_dir=%s\nsys.path=%s' % (prototype_dir, learning_dir, sys.path))

prototype_dir=/Users/makmorit/GitHub/donusagi-bot/learning/prototype
learning_dir=/Users/makmorit/GitHub/donusagi-bot/learning
sys.path=['', '/Library/Frameworks/Python.framework/Versions/3.5/lib/python35.zip', '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5', '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/plat-darwin', '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/lib-dynload', '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages', '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/extensions', '/Users/makmorit/.ipython', '/Users/makmorit/GitHub/donusagi-bot/learning']


In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定したものを使用します。
'''
csv_file_name = 'test_daikin_conversation.csv'
original_csv_dir = os.path.join(learning_dir, 'learning/tests/engine/fixtures/')
original_file_path = os.path.join(original_csv_dir, csv_file_name)

csv_dir = os.path.join(prototype_dir, 'resources')

import shutil
shutil.copy2(original_file_path, csv_dir)
copied_csv_file_path = os.path.join(csv_dir, csv_file_name)

print('CSV file for test=[%s]' % copied_csv_file_path)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]


## (2) 既存モジュールをカスタマイズ

In [3]:
'''
    Bot/Reply モジュールをカスタマイズした
    BotForLocalTest/ReplyForLocalTest モジュールは、
    {prototype_dir}/modules 配下に格納されています
    （ローカル環境から MySQLdb/dataset に接続できないための措置）
'''
from prototype.modules.BotForLocalTest import Bot
from prototype.modules.ReplyForLocalTest import Reply
from learning.core.learn.learning_parameter import LearningParameter

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
bot_id = 8888
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)
csv_file_path = copied_csv_file_path
csv_file_encoding = 'utf-8'

## (3) 交差検証実行前の調査

scikit-learn の cross_validation.py においては、クラスにおけるサンプル数が、交差検証（K-fold）における分割数（デフォルト＝３件）を下回っていないかどうかチェックします。

下回っている場合、当該クラスについては、交差検証の結果をそのまま適用できないため、警告を表示しているものと考えられます。

ただし、cross_validation.py が警告を出しても、そのまま処理を続行してしまうところが「？？？」といった感じではあります。

In [5]:
'''
    訓練データの生成（内部で TF-IDF 処理を実行）
'''
from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv
training_set = TrainingMessageFromCsv(bot_id, csv_file_path, learning_parameter, encoding=csv_file_encoding)
build_training_set_from_csv = training_set.build()

TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017/03/02 PM 06:01:45 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
TextArray#__init__ start
2017/03/02 PM 06:01:45 TextArray#__init__ start
TextArray#to_vec start
2017/03/02 PM 06:01:45 TextArray#to_vec start
TextArray#to_vec end
2017/03/02 PM 06:02:03 TextArray#to_vec end
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
2017/03/02 PM 06:02:03 [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


### 訓練データのクラス（ここでは、回答ID）を抽出

In [6]:
'''
    訓練データ分割前のチェック処理
    
    scikit-learn/sklearn/cross_validation.py 内部の処理を切り出して実行
'''
import numpy as np
y = np.asarray(build_training_set_from_csv.y)
n_samples = y.shape[0]
unique_labels, y_inversed = np.unique(y, return_inverse=True)
unique_labels

array([3397, 3398, 3399, 3400, 3401, 3402, 3404, 3405, 3406, 3407, 3408,
       3409, 3411, 3412, 3413, 3414, 3415, 3416, 3417, 3418, 3419, 3420,
       3421, 3422, 3423, 3424, 3425, 3426, 3427, 3428, 3429, 3430, 3431,
       3432, 3433, 3434, 3435, 3436, 3437, 3438, 3439, 3440, 3441, 3442,
       3443, 3444, 3445, 3446, 3447, 3449, 3451, 3452, 3453, 3454, 3455,
       3456, 3457, 3458, 3459, 3460, 3461, 3462, 3463, 3464, 3465, 3466,
       3467, 3468, 3469, 3470, 3471, 3472, 3473, 3474, 3475, 3476, 3477,
       3478, 3479, 3480, 3481, 3482, 3483, 3484, 3485, 3486, 3487, 3488,
       3489, 3490, 3491, 3492, 3493, 3494, 3496, 3497, 3498, 3499, 3500,
       3501, 3502, 3503, 3504, 3505, 3506, 3507, 3508, 3509, 3510, 3511,
       3512, 3513, 3514, 3515, 3516, 3517, 3518, 3519, 3520, 3521, 3522,
       3523, 3524, 3525, 3526, 3527, 3528, 3529, 3530, 3531, 3532, 3534,
       3535, 3536, 3537, 3538, 3539, 3540, 3541, 3543, 3544, 3545, 3546,
       3547, 3548, 3549, 3550, 3551, 3552, 3553, 35

### 訓練データのクラスごとに含まれるサンプル（ここでは質問文）の数を取得

In [7]:
def bincount(x, weights=None, minlength=None):
    if len(x) > 0:
        return np.bincount(x, weights, minlength) # <---
    else:
        if minlength is None:
            minlength = 0
        minlength = np.asscalar(np.asarray(minlength, dtype=np.intp))
        return np.zeros(minlength, dtype=np.intp)

label_counts = bincount(y_inversed)
label_counts

array([24, 54, 30, 26, 42, 42, 44, 42, 36, 36, 36, 42, 26, 30, 36, 32, 38,
       32, 32, 30, 42, 32, 36, 46, 38, 42, 36, 38, 32, 36, 38, 30, 36, 38,
       54, 38, 32, 52, 30, 38, 38, 26, 30, 30, 32, 32, 36, 36, 32, 30, 36,
       38, 26, 42, 42, 42, 36, 36, 30, 42, 48, 36, 26, 36, 38, 24, 36, 32,
       36, 36, 32, 30, 32, 38, 36, 20, 66, 38, 38, 40, 26, 42, 42, 42, 46,
       42, 42, 48, 26, 32, 36, 32, 36, 38, 36, 49, 24, 20, 38, 28, 34, 38,
       32, 34, 32, 32, 38, 26, 26, 38, 32, 32, 32, 28, 32, 38, 82, 32, 32,
       26, 36, 36, 26, 26, 32, 62, 32, 40, 32, 34, 32, 32, 38, 26, 26, 38,
       32, 32, 32, 32, 38, 32, 38, 38, 34, 32, 26, 44, 32, 32, 32, 32, 26,
       34, 38, 26, 38, 32, 32, 32, 28, 38, 34, 38, 32, 32, 32, 38, 38, 22,
       26, 32, 32, 38, 26, 26, 32, 32, 38, 26, 38, 38, 38, 46, 32, 34, 40,
       32, 32, 26, 32, 56, 38, 38, 26, 38, 32, 26, 32, 26, 32, 40, 32, 32,
       32, 28, 32, 32, 32, 38, 26, 32, 38, 38, 32, 28, 44, 32, 38, 34, 32,
       38, 34, 44, 26, 32

### クラスに含まれるサンプルの数が、訓練データ分割数を下回っていないかチェック

In [8]:
min_labels = np.min(label_counts) # <---ここの値が２になったために警告が出た様子

n_folds = 3 # これは K-fold 交差検証のデフォルト

if n_folds > min_labels:
    import warnings
    warnings.warn(("The least populated class in y has only %d"
                   " members, which is too few. The minimum"
                   " number of labels for any class cannot"
                   " be less than n_folds=%d."
                   % (min_labels, n_folds)), Warning)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:11: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.


### ご参考：クラスのサンプル数＜分割数になっているクラスIDの一覧

In [9]:
warning_class_ids = []
for i, unique_label in enumerate(unique_labels):
    if label_counts[i] < n_folds:
        warning_class_ids.append([unique_label, label_counts[i]])

warning_class_ids

[[3931, 2],
 [3933, 2],
 [3935, 2],
 [3938, 2],
 [3973, 2],
 [3976, 2],
 [3977, 2],
 [3979, 2],
 [3981, 2],
 [3984, 2],
 [3988, 2],
 [3989, 2],
 [3991, 2],
 [3993, 2],
 [3994, 2],
 [3995, 2],
 [3996, 2],
 [3997, 2],
 [3998, 2],
 [4000, 2],
 [4003, 2],
 [4005, 2],
 [4644, 2],
 [4645, 2],
 [4646, 2],
 [4647, 2],
 [4649, 2],
 [4650, 2],
 [4651, 2],
 [4652, 2],
 [4653, 2],
 [4654, 2],
 [4655, 2],
 [4656, 2],
 [4659, 2],
 [4660, 2],
 [4661, 2],
 [4663, 2],
 [4666, 2],
 [4668, 2],
 [4669, 2],
 [4670, 2],
 [4672, 2]]

### 参考文献：

- scikit-learn の cross_validation.py のコード

 https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/cross_validation.py
 

- numpy 関数のマニュアル

 https://docs.scipy.org/doc/numpy/reference/generated/numpy.unique.html#numpy.unique

 https://docs.scipy.org/doc/numpy/reference/generated/numpy.bincount.html